# RAPTOR

---

<br>

Run command:

```python pyraptor/gtfs/timetable.py -i "data/input/NL-gtfs" -o "data/output" -d "20220517" -a NS --icd```

## Setup

In [1]:
input_folder: str = "data/input/NL-gtfs"
output_folder: str = "data/output"
departure_date: str = "20220517"
agencies: list[str] = ['NS']
icd_fix: bool = False

## Imports

In [2]:
from collections import defaultdict
import pandas as pd
import os

In [3]:
from pyraptor.dao import write_timetable
from pyraptor.util import mkdir_if_not_exists, str2sec, TRANSFER_COST
from pyraptor.model.structures import (
    Timetable,
    Stop,
    Stops,
    Trip,
    Trips,
    TripStopTime,
    TripStopTimes,
    Station,
    Stations,
    Routes,
    Transfer,
    Transfers,
)

## Timetable

- GTFSTimetable
- pyRAPTORTimetable

### GTFSTimetable

In [4]:
# @dataclass
class GtfsTimetable:
    """Gtfs Timetable data"""

    trips = None
    calendar = None
    stop_times = None
    stops = None

In [5]:
def read_gtfs_timetable(
    input_folder: str, departure_date: str, agencies: list[str]
) -> GtfsTimetable:
    """Extract operators from GTFS data"""

    # logger.info("Read GTFS data")


    # AGENCY.TXT
    # logger.debug("Read Agencies")

    # Load dataframe and slice to specified agencies
    agencies_df = pd.read_csv(os.path.join(input_folder, "agency.txt"))
    agencies_df = agencies_df.loc[agencies_df["agency_name"].isin(agencies)][
        ["agency_id", "agency_name"]
    ]
    agency_ids = agencies_df.agency_id.values
    assert len(agency_ids) > 0
    # logger.debug(f"Agencies found: {agency_ids}")


    # ROUTES.TXT
    # logger.debug("Read Routes")

    # Get routes by agency
    routes = pd.read_csv(os.path.join(input_folder, "routes.txt"))
    routes = routes[routes.agency_id.isin(agency_ids)]
    routes = routes[
        ["route_id", "agency_id", "route_short_name", "route_long_name", "route_type"]
    ]
    assert len(routes) > 0
    # logger.debug(f"Routes found: {len(routes)} routes")


    # TRIPS.TXT
    # logger.debug("Read Trips")

    # Get trips in routes
    trips = pd.read_csv(os.path.join(input_folder, "trips.txt"))
    trips = trips[trips.route_id.isin(routes.route_id.values)]
    trips = trips[
        [
            "route_id",
            "service_id",
            "trip_id",
            "trip_short_name",
            "trip_long_name",
        ]
    ]
    trips["trip_short_name"] = trips["trip_short_name"].astype(int)
    assert len(trips) > 0
    # logger.debug(f"Trips found: {len(trips)} trips")


    # CALENDAR.TXT
    # logger.debug("Read Calendar")

    # Get calendar from service_id of trips (correct dates)
    calendar = pd.read_csv(
        os.path.join(input_folder, "calendar_dates.txt"), dtype={"date": str}
    )
    assert departure_date in calendar.date.values
    # logger.debug(f"Departure date ({departure_date}) in GTFS ({input_folder + 'calendar_dates.txt'})")
    calendar = calendar[calendar.service_id.isin(trips.service_id.values)]

    # Add date to trips and filter on departure date
    trips = trips.merge(calendar[["service_id", "date"]], on="service_id")
    trips = trips[trips.date == departure_date]
    assert len(trips) > 0
    # logger.debug(f"Trips on departure date found: {len(trips)} trips")


    # STOP_TIMES.TXT
    # logger.debug("Read Stop Times")

    # Get stop times of trips
    stop_times = pd.read_csv(
        os.path.join(input_folder, "stop_times.txt"), dtype={"stop_id": str}
    )
    stop_times = stop_times[stop_times.trip_id.isin(trips.trip_id.values)]
    stop_times = stop_times[
        [
            "trip_id",
            "stop_sequence",
            "stop_id",
            "arrival_time",
            "departure_time",
        ]
    ]
    # Convert times to seconds past midnight
    stop_times["arrival_time"] = stop_times["arrival_time"].apply(str2sec)
    stop_times["departure_time"] = stop_times["departure_time"].apply(str2sec)
    assert len(stop_times) > 0
    # logger.debug(f"Stop times found: {len(stop_times)} stop times")


    # PLATFORMS (STOPS.TXT)
    # NOTE: Platforms are found from unique `stop_id`s
    # logger.debug("Read Stops")

    stops_full = pd.read_csv(
        os.path.join(input_folder, "stops.txt"), dtype={"stop_id": str}
    )
    stops = stops_full.loc[
        stops_full["stop_id"].isin(stop_times.stop_id.unique())
    ].copy()

    # Read stopareas, i.e. stations
    stopareas = stops["parent_station"].unique()
    # stops = stops.append(.copy())
    stops = pd.concat([stops, stops_full.loc[stops_full["stop_id"].isin(stopareas)]])

    # stops["zone_id"] = stops["zone_id"].str.replace("IFF:", "").str.upper()
    stops["stop_code"] = stops.stop_code.str.upper()
    stops = stops[
        [
            "stop_id",
            "stop_name",
            "parent_station",
            "platform_code",
        ]
    ]

    # Filter out the general station codes
    stops = stops.loc[~stops.parent_station.isna()]
    assert len(stops) > 0
    stops = stops.loc[~stops.parent_station.isna()]

    gtfs_timetable = GtfsTimetable()
    gtfs_timetable.trips = trips
    gtfs_timetable.stop_times = stop_times
    gtfs_timetable.stops = stops

    return gtfs_timetable

In [6]:
gtfs_timetable = read_gtfs_timetable(
    input_folder=input_folder,
    departure_date=departure_date,
    agencies=agencies)

### PyRAPTORTimetable

In [7]:
def gtfs_to_pyraptor_timetable(
    gtfs_timetable: GtfsTimetable, icd_fix: bool = False
) -> Timetable:
    """
    Convert timetable for usage in Raptor algorithm.
    """
    # logger.info("Convert GTFS timetable to timetable for PyRaptor algorithm")

    # Stations and stops, i.e. platforms
    # logger.debug("Add stations and stops")

    stations = Stations()
    stops = Stops()

    gtfs_timetable.stops.platform_code = gtfs_timetable.stops.platform_code.fillna("?")

    for s in gtfs_timetable.stops.itertuples():
        station = Station(s.stop_name, s.stop_name)
        station = stations.add(station)

        stop_id = f"{s.stop_name}-{s.platform_code}"
        stop = Stop(s.stop_id, stop_id, station, s.platform_code)

        station.add_stop(stop)
        stops.add(stop)

    # Stop Times
    stop_times = defaultdict(list)
    for stop_time in gtfs_timetable.stop_times.itertuples():
        stop_times[stop_time.trip_id].append(stop_time)

    # Trips and Trip Stop Times
    # logger.debug("Add trips and trip stop times")

    trips = Trips()
    trip_stop_times = TripStopTimes()

    for trip_row in gtfs_timetable.trips.itertuples():
        trip = Trip()
        trip.hint = trip_row.trip_short_name  # i.e. treinnummer
        trip.long_name = trip_row.trip_long_name  # e.g., Sprinter

        # Iterate over stops
        sort_stop_times = sorted(
            stop_times[trip_row.trip_id], key=lambda s: int(s.stop_sequence)
        )
        for stopidx, stop_time in enumerate(sort_stop_times):
            # Timestamps
            dts_arr = stop_time.arrival_time
            dts_dep = stop_time.departure_time

            # Trip Stop Times
            stop = stops.get(stop_time.stop_id)

            # GTFS files do not contain ICD supplement fare, so hard-coded here
            fare = calculate_icd_fare(trip, stop, stations) if icd_fix is True else 0
            trip_stop_time = TripStopTime(trip, stopidx, stop, dts_arr, dts_dep, fare)

            trip_stop_times.add(trip_stop_time)
            trip.add_stop_time(trip_stop_time)

        # Add trip
        if trip:
            trips.add(trip)

    # Routes
    # logger.debug("Add routes")

    routes = Routes()
    for trip in trips:
        routes.add(trip)

    # Transfers
    # logger.debug("Add transfers")

    transfers = Transfers()
    for station in stations:
        station_stops = station.stops
        station_transfers = [
            Transfer(from_stop=stop_i, to_stop=stop_j, layovertime=TRANSFER_COST)
            for stop_i in station_stops
            for stop_j in station_stops
            if stop_i != stop_j
        ]
        for st in station_transfers:
            transfers.add(st)

    # Timetable
    timetable = Timetable(
        stations=stations,
        stops=stops,
        trips=trips,
        trip_stop_times=trip_stop_times,
        routes=routes,
        transfers=transfers,
    )
    timetable.counts()

    return timetable

In [8]:
timetable = gtfs_to_pyraptor_timetable(
    gtfs_timetable=gtfs_timetable,
    icd_fix=icd_fix)

2022-07-18 12:43:07.091 | DEBUG    | pyraptor.model.structures:counts:41 - Counts:
2022-07-18 12:43:07.092 | DEBUG    | pyraptor.model.structures:counts:42 - Stations   : 248
2022-07-18 12:43:07.093 | DEBUG    | pyraptor.model.structures:counts:43 - Routes     : 899
2022-07-18 12:43:07.093 | DEBUG    | pyraptor.model.structures:counts:44 - Trips      : 4305
2022-07-18 12:43:07.093 | DEBUG    | pyraptor.model.structures:counts:45 - Stops      : 736
2022-07-18 12:43:07.094 | DEBUG    | pyraptor.model.structures:counts:46 - Stop Times : 42508
2022-07-18 12:43:07.094 | DEBUG    | pyraptor.model.structures:counts:47 - Transfers  : 3036


In [9]:
timetable

Timetable(stations=<Stations(n_stations=248)>, stops=Stops(n_stops=736), trips=Trips(n_trips=4305), trip_stop_times=TripStoptimes(n_tripstoptimes=42508), routes=Routes(n_routes=899), transfers=Transfers(n_transfers=3036))

# McRaptor

## Imports

In [10]:
from typing import List, Tuple, Dict
from copy import copy
from time import perf_counter

from loguru import logger
from pyraptor.model.structures import (
    Timetable,
    Stop,
    Route,
    Bag,
    Label,
    Leg,
    Journey,
    pareto_set,
)

## McRaptorAlgorithm

(UNCHANGED)

In [11]:
class McRaptorAlgorithm:
    """McRAPTOR Algorithm ***Remains UNCHANGED in phiRAPTOR implementation.***"""

    def __init__(self, timetable: Timetable):
        self.timetable = timetable

    def run(
        self, from_stops: List[Stop], dep_secs: int, rounds: int, previous_run: Dict[int, Bag] = None
    ) -> Dict[int, Dict[int, Bag]]:
        """Run Round-Based Algorithm"""

        s = perf_counter()

        # Initialize empty bag, i.e. B_k(p) = [] for every k and p
            # The bag, B_k(p), stores non-dominating labels, L, for each stop, p, in round k
                # Is this the id of the trip which dominates? No.
                    # It is a class with multiple attributes,
                    # E.g. earliest_arrival_time, fare, n_trips (defined in criteria)
        bag_round_stop: Dict[int, Dict[Stop, Bag]] = {}
        for k in range(0, rounds + 1):
            bag_round_stop[k] = {}
            for p in self.timetable.stops:
                bag_round_stop[k][p] = Bag()

        # Add origin stops to bag
        logger.debug(f"Starting from Stop IDs: {str(from_stops)}")

        # Initialize bag for round 0, i.e. add Labels with criterion 0 for all from stops
        if previous_run != None:
            # For the range query
            bag_round_stop[0] = copy(previous_run)

        for from_stop in from_stops:
            # bag_round_stop[0][from_stop].add(Label(dep_secs, 0, None, from_stop))
            bag_round_stop[0][from_stop].add(Label(dep_secs, 0, None, from_stop, 0))

        marked_stops = from_stops

        # Run rounds
        actual_rounds = 0
        for k in range(1, rounds + 1):
            logger.info(f"Analyzing possibilities round {k}")
            logger.debug(f"Stops to evaluate count: {len(marked_stops)}")

            # Copy bag from previous round
            bag_round_stop[k] = copy(bag_round_stop[k - 1])

            if len(marked_stops) > 0:
                actual_rounds = k

                # Accumulate routes serving marked stops from previous round
                route_marked_stops = self.accumulate_routes(marked_stops)

                # Traverse each route
                bag_round_stop, marked_stops_trips = self.traverse_route(
                    bag_round_stop, k, route_marked_stops
                )

                # Now add footpath transfers and update
                bag_round_stop, marked_stops_transfers = self.add_transfer_time(
                    bag_round_stop, k, marked_stops_trips
                )

                marked_stops = set(marked_stops_trips).union(marked_stops_transfers)
            else:
                break

        logger.info("Finish round-based algorithm to create bag with best labels")
        logger.info(f"Running time: {perf_counter() - s}")

        return bag_round_stop, actual_rounds

    def accumulate_routes(self, marked_stops) -> List[Tuple[Route, Stop]]:
        """Accumulate routes serving marked stops from previous round, i.e. Q"""
        route_marked_stops = {}  # i.e. Q
        for marked_stop in marked_stops:
            routes_serving_stop = self.timetable.routes.get_routes_of_stop(marked_stop)
            for route in routes_serving_stop:
                # Check if new_stop is before existing stop in Q
                current_stop_for_route = route_marked_stops.get(route, None)  # p'
                if (current_stop_for_route is None) or (
                    route.stop_index(current_stop_for_route)
                    > route.stop_index(marked_stop)
                ):
                    route_marked_stops[route] = marked_stop
        route_marked_stops = [(r, p) for r, p in route_marked_stops.items()]

        logger.debug(f"Found {len(route_marked_stops)} routes serving marked stops")

        return route_marked_stops

    def traverse_route(
        self,
        bag_round_stop: Dict[int, Dict[int, Bag]],
        k: int,
        route_marked_stops: List[Tuple[Route, Stop]],
    ) -> Tuple[Dict[int, Dict[int, Bag]], List[Stop]]:
        """
        Traverse through all marked route-stops and update labels accordingly.

        :param bag_round_stop: Bag per round per stop
        :param k: current round
        :param route_marked_stops: list of marked (route, stop) for evaluation

        OCCUPANCY:
        To update occupancy for this label, we will need access to previous trips/legs.
        """

        new_marked_stops = set()

        for (marked_route, marked_stop) in route_marked_stops:
            # Traversing through route from marked stop
            route_bag = Bag()

            # Get all stops after current stop within the current route
            marked_stop_index = marked_route.stop_index(marked_stop)
            remaining_stops_in_route = marked_route.stops[marked_stop_index:]

            # For the rest of the trips (stops) on this route
                # Add the occupancy for this trip
            for stop_idx, current_stop in enumerate(remaining_stops_in_route):

                # Step 1: update earliest arrival times and criteria for each label L in route-bag
                update_labels = []
                for label in route_bag.labels:
                    trip_stop_time = label.trip.get_stop(current_stop)

                    # Take fare of previous stop in trip as fare is defined on start
                    # Take OCCUPANCY from previous stop in trip as occupancy defined on start
                    previous_stop = remaining_stops_in_route[stop_idx - 1]
                    from_fare = label.trip.get_fare(previous_stop)
                    from_occupancy = label.trip.get_stop_occupancy(previous_stop) # 06/06/2022

                    label = label.update(
                        earliest_arrival_time=trip_stop_time.dts_arr,
                        fare_addition=from_fare,
                        occupancy_addition=from_occupancy,
                    )

                    update_labels.append(label)
                route_bag = Bag(labels=update_labels)

                # Step 2: merge bag_route into bag_round_stop and remove dominated labels
                # The label contains the trip with which one arrives at current stop with k legs
                # and we boarded the trip at from_stop.
                bag_round_stop[k][current_stop] = bag_round_stop[k][current_stop].merge(
                    route_bag
                )
                bag_update = bag_round_stop[k][current_stop].update

                # Mark stop if bag is updated
                if bag_update:
                    new_marked_stops.add(current_stop)

                # Step 3: merge B_{k-1}(p) into B_r
                route_bag = route_bag.merge(bag_round_stop[k - 1][current_stop])

                # Assign trips to all newly added labels in route_bag
                # This is the trip on which we board
                update_labels = []
                for label in route_bag.labels:
                    earliest_trip = marked_route.earliest_trip(
                        label.earliest_arrival_time, current_stop
                    )
                    if earliest_trip is not None:
                        # Update label with earliest trip in route leaving from this station
                        # If trip is different we board the trip at current_stop
                        label = label.update_trip(earliest_trip, current_stop)
                        update_labels.append(label)
                route_bag = Bag(labels=update_labels)

        logger.debug(f"{len(new_marked_stops)} reachable stops added")

        return bag_round_stop, new_marked_stops

    def add_transfer_time(
        self,
        bag_round_stop: Dict[int, Dict[Stop, Bag]],
        k: int,
        marked_stops: List[Stop],
    ) -> Tuple:
        """Add transfers between platforms."""

        marked_stops_transfers = set()

        # Add in transfers to other platforms
        for stop in marked_stops:
            other_station_stops = [st for st in stop.station.stops if st != stop]

            for other_stop in other_station_stops:
                # Create temp copy of B_k(p_i)
                temp_bag = Bag()
                for label in bag_round_stop[k][stop].labels:
                    # Add arrival time to each label
                    transfer_arrival_time = (
                        label.earliest_arrival_time
                        + self.get_transfer_time(stop, other_stop)
                    )
                    # Update label with new earliest arrival time at other_stop
                    label = label.update(
                        earliest_arrival_time=transfer_arrival_time,
                        fare_addition=0,
                        from_stop=stop,
                    )
                    temp_bag.add(label)

                # Merg temp bag into B_k(p_j)
                bag_round_stop[k][other_stop] = bag_round_stop[k][other_stop].merge(
                    temp_bag
                )
                bag_update = bag_round_stop[k][other_stop].update

                # Mark stop if bag is updated
                if bag_update:
                    marked_stops_transfers.add(other_stop)

        logger.debug(f"{len(marked_stops_transfers)} transferable stops added")

        return bag_round_stop, marked_stops_transfers

    def get_transfer_time(self, stop_from: Stop, stop_to: Stop) -> int:
        """
        Calculate the transfer time from a stop to another stop (usually at one station)
        """
        transfers = self.timetable.transfers
        return transfers.stop_to_stop_idx[(stop_from, stop_to)].layovertime


## best_legs_to_destination_station

(UNCHANGED)

In [12]:
# Implementation of `bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)`

def best_legs_to_destination_station(
    to_stops: List[Stop], last_round_bag: Dict[Stop, Bag]
) -> List[Leg]:
    """
    Find the last legs to destination station that are reached by non-dominated labels.
    ***Remains UNCHANGED in phiRAPTOR implementation.***
    """

    # Find all labels to target_stops
    best_labels = [
        (stop, label) for stop in to_stops for label in last_round_bag[stop].labels
    ]

    # TODO Use merge function on Bag
    # Pareto optimal labels
    pareto_optimal_labels = pareto_set([label for (_, label) in best_labels])
    pareto_optimal_labels = [
        (stop, label) for (stop, label) in best_labels if label in pareto_optimal_labels
    ]

    # Label to leg, i.e. add to_stop
    legs = [
        Leg(
            label.from_stop, # from_stop: Stop
            to_stop, # to_stop: Stop
            label.trip, # trip: Trip
            label.earliest_arrival_time, # earliest_arrival_time: int
            label.fare, # fare: int = 0
            # label.occupancy, # occupancy: int = 0 # 03/06/2022
            label.n_trips, # n_trips: int = 0
        )
        for (to_stop, label) in pareto_optimal_labels
    ]
    return legs

## query_mcraptor

### Imports

In [13]:
"""Run query with RAPTOR algorithm"""
import argparse
from typing import List, Dict
from copy import copy
from time import perf_counter

from loguru import logger

from pyraptor.dao.timetable import read_timetable
from pyraptor.model.structures import Timetable, Journey, Station
# from pyraptor.model.mcraptor import (
#     McRaptorAlgorithm,
#     reconstruct_journeys,
#     best_legs_to_destination_station,
# )
from pyraptor.util import str2sec

### pick_journey

(NEW)

In [14]:
def pick_journey(journeys):
    """Return the journey corresponsing to the lowest journey_cost"""
    # return journeys[0] if len(journeys) == 0 else journeys[journeys.index(min(journeys))]
    return journeys[0] if len(journeys) == 0 else journeys[[journey.journey_cost() for journey in journeys].index(min([journey.journey_cost() for journey in journeys]))]

### main

(CHANGED)

In [15]:
def main(
    input_folder,
    origin_station,
    destination_station,
    departure_time,
    rounds,
    track_occupancy: bool = True,
):
    """Run RAPTOR algorithm. ***Is CHANGED in phiRAPTOR implementation.***"""

    logger.debug("Input directory     : {}", input_folder)
    logger.debug("Origin station      : {}", origin_station)
    logger.debug("Destination station : {}", destination_station)
    logger.debug("Departure time      : {}", departure_time)
    logger.debug("Rounds              : {}", str(rounds))

    # timetable = read_timetable(input_folder)

    logger.info(f"Calculating network from : {origin_station}")

    # Departure time seconds
    dep_secs = str2sec(departure_time)
    logger.debug("Departure time (s.)  : " + str(dep_secs))

    # Find route between two stations
    journeys_to_destinations = run_mcraptor(
        timetable,
        origin_station,
        dep_secs,
        rounds,
    )

    # Output journey
    journeys = journeys_to_destinations[destination_station]
    logger.info(f"Number of journeys to destination: {len(journeys)}") # 13/06/2022
    if len(journeys) != 0:
        for jrny in journeys:
            jrny.print(dep_secs=dep_secs)

    ### CHANGES for phiRAPTOR:
    journey = pick_journey(journeys)

    if track_occupancy:
        for leg in journey.legs:
            leg.trip.update_occupancy(leg.from_stop)

    return journey

### reconstruct_journeys

(UNCHANGED)

In [16]:
def reconstruct_journeys(
    from_stops: List[Stop],
    destination_legs: List[Leg],
    bag_round_stop: Dict[int, Dict[Stop, Bag]],
    k: int,
) -> List[Journey]:
    """
    Construct Journeys for destinations from bags by recursively
    looping from destination to origin.
    ***Remains UNCHANGED in phiRAPTOR implementation.***
    """

    def loop(
        bag_round_stop: Dict[int, Dict[Stop, Bag]], k: int, journeys: List[Journey]
    ):
        """Create full journey by prepending legs recursively"""

        last_round_bags = bag_round_stop[k]

        for jrny in journeys:
            current_leg = jrny[0]

            # End of journey if we are at origin stop or journey is not feasible
            if current_leg.trip is None or current_leg.from_stop in from_stops:
                jrny = jrny.remove_transfer_legs()
                if jrny.is_valid() is True:
                    yield jrny
                continue

            ###################################################################

            # HERE - Update occupancy

            ###################################################################
            # Loop trough each new leg
            labels_to_from_stop = last_round_bags[current_leg.from_stop].labels
            for new_label in labels_to_from_stop:

                """
                If we call the new_label.update() method here,
                we would be updating the Label, not the TripStopTime.
                Hence, it would be reset every time we do a new search.

                Instead, we need to update the TripStopTime occupancy.
                We do this by getting the TripStopTime instance by calling the
                trip.get_stop method and calling our custom update_occupancy
                method on the resulting TripStopTime instance.

                CHECK:
                We should only instanctiate occupancy as 0 ONCE,
                during the initialisation of the TripStopTime instance.
                Thereonin, we should intitialise occupancy to None,
                and/or inherit from the TripStopTime occupancy.
                """

                # new_label.trip.get_stop_occupancy()
                # print(f"new_label.trip: {new_label.trip}")
                # new_label.trip.get_stop(current_leg.from_stop).update_occupancy()
                # new_label.trip.update_occupancy(current_leg.from_stop) # 08/06/2022

                """We don't want to update occupancy here, this does it for all journeys from origin.
                We only want to do so between the chosen journey"""

                new_leg = Leg(
                    new_label.from_stop, # from_stop: Stop
                    current_leg.from_stop, # to_stop: Stop
                    new_label.trip, # trip: Trip
                    new_label.earliest_arrival_time, # earliest_arrival_time: int
                    new_label.fare, # fare: int = 0
                    # new_label.occupancy, # occupancy: int = 0 # 03/06/2022
                    new_label.n_trips, # n_trips: int = 0
                )
                # Only add new_leg if compatible before current leg, e.g. earlier arrival time, etc.
                if new_leg.is_compatible_before(current_leg):
                    new_jrny = jrny.prepend_leg(new_leg)
                    for i in loop(bag_round_stop, k, [new_jrny]):
                        yield i

    journeys = [Journey(legs=[leg]) for leg in destination_legs]
    journeys = [jrny for jrny in loop(bag_round_stop, k, journeys)]

    return journeys

### run_mcraptor

(UNCHANGED)

In [17]:
def run_mcraptor(
    timetable: Timetable,
    origin_station: str,
    dep_secs: int,
    rounds: int,
) -> Dict[Station, List[Journey]]:
    """
    Perform the McRaptor algorithm.

    :param timetable: timetable
    :param origin_station: Name of origin station
    :param dep_secs: Time of departure in seconds
    :param rounds: Number of iterations to perform
    
    ***Remains UNCHANGED in phiRAPTOR implementation.***
    """

    # Run Round-Based Algorithm for an origin station
    from_stops = timetable.stations.get(origin_station).stops
    raptor = McRaptorAlgorithm(timetable)
    bag_round_stop, actual_rounds = raptor.run(from_stops, dep_secs, rounds)
    last_round_bag = copy(bag_round_stop[rounds])

    # Calculate journets to all destinations
    logger.info("Calculating journeys to all destinations")
    s = perf_counter()

    destination_stops = {
        st.name: timetable.stations.get_stops(st.name) for st in timetable.stations
    }
    destination_stops.pop(origin_station, None)

    journeys_to_destinations = dict()
    for destination_station_name, to_stops in destination_stops.items():
        destination_legs = best_legs_to_destination_station(to_stops, last_round_bag)

        if len(destination_legs) == 0:
            logger.info("Destination unreachable with given parameters")
            continue

        journeys = reconstruct_journeys(
            from_stops, destination_legs, bag_round_stop, k=rounds
        )
        journeys_to_destinations[destination_station_name] = journeys

    logger.info(f"Journey calculation time: {perf_counter() - s}")
    # logger.info(f"Number of journeys to destination: {len(journeys_to_destinations)}") # 13/06/2022

    return journeys_to_destinations

### Arguments

In [18]:
# if __name__ == "__main__":
#     args = parse_arguments()
#     main(
#         args.input,
#         args.origin,
#         args.destination,
#         args.time,
#         args.rounds,
#     )

In [19]:
input_folder = "data/output"
origin_station = "Arnhem Zuid"
destination_station = "Oosterbeek"
departure_time = "08:30:00"
rounds = 5

### query_raptor

To query raptor, we must run `main()`

In [20]:
main(
    input_folder,
    origin_station,
    destination_station,
    departure_time,
    rounds,
)

2022-07-18 12:43:07.587 | DEBUG    | __main__:main:11 - Input directory     : data/output
2022-07-18 12:43:07.588 | DEBUG    | __main__:main:12 - Origin station      : Arnhem Zuid
2022-07-18 12:43:07.588 | DEBUG    | __main__:main:13 - Destination station : Oosterbeek
2022-07-18 12:43:07.589 | DEBUG    | __main__:main:14 - Departure time      : 08:30:00
2022-07-18 12:43:07.590 | DEBUG    | __main__:main:15 - Rounds              : 5
2022-07-18 12:43:07.590 | INFO     | __main__:main:19 - Calculating network from : Arnhem Zuid
2022-07-18 12:43:07.590 | DEBUG    | __main__:main:23 - Departure time (s.)  : 30600
2022-07-18 12:43:07.595 | DEBUG    | __main__:run:26 - Starting from Stop IDs: [Stop(Arnhem Zuid-1 [2323103]), Stop(Arnhem Zuid-2 [2323104])]
2022-07-18 12:43:07.596 | INFO     | __main__:run:42 - Analyzing possibilities round 1
2022-07-18 12:43:07.597 | DEBUG    | __main__:run:43 - Stops to evaluate count: 2
2022-07-18 12:43:07.597 | DEBUG    | __main__:accumulate_routes:88 - Foun

Journey(n_legs=2)

In [21]:
main(
    input_folder,
    origin_station,
    destination_station,
    departure_time,
    rounds,
)

2022-07-18 12:43:09.703 | DEBUG    | __main__:main:11 - Input directory     : data/output
2022-07-18 12:43:09.704 | DEBUG    | __main__:main:12 - Origin station      : Arnhem Zuid
2022-07-18 12:43:09.706 | DEBUG    | __main__:main:13 - Destination station : Oosterbeek
2022-07-18 12:43:09.707 | DEBUG    | __main__:main:14 - Departure time      : 08:30:00
2022-07-18 12:43:09.707 | DEBUG    | __main__:main:15 - Rounds              : 5
2022-07-18 12:43:09.708 | INFO     | __main__:main:19 - Calculating network from : Arnhem Zuid
2022-07-18 12:43:09.709 | DEBUG    | __main__:main:23 - Departure time (s.)  : 30600
2022-07-18 12:43:09.714 | DEBUG    | __main__:run:26 - Starting from Stop IDs: [Stop(Arnhem Zuid-1 [2323103]), Stop(Arnhem Zuid-2 [2323104])]
2022-07-18 12:43:09.715 | INFO     | __main__:run:42 - Analyzing possibilities round 1
2022-07-18 12:43:09.715 | DEBUG    | __main__:run:43 - Stops to evaluate count: 2
2022-07-18 12:43:09.716 | DEBUG    | __main__:accumulate_routes:88 - Foun

Journey(n_legs=2)